In [2]:
from sklearn.datasets import fetch_openml

ModuleNotFoundError: No module named 'sklearn'

In [7]:
mnist = fetch_openml('mnist_784', version=1)        

In [9]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])